<a href="https://colab.research.google.com/github/ANSUNMIN98/Saju/blob/main/20250926_%EC%B5%9C%EC%A2%85%EB%AA%85%EB%B0%98JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
from collections import Counter
import pandas as pd

# ======================
# 천간/지지/오행/음양 정의
# ======================
stems = ["갑","을","병","정","무","기","경","신","임","계"]
branches = ["자","축","인","묘","진","사","오","미","신","유","술","해"]
sexagenary = [stems[i % 10] + branches[i % 12] for i in range(60)]

element_map = {
    "갑":"목", "을":"목", "병":"화", "정":"화", "무":"토",
    "기":"토", "경":"금", "신":"금", "임":"수", "계":"수"
}
yin_yang_map = {
    "갑":"양", "을":"음", "병":"양", "정":"음", "무":"양",
    "기":"음", "경":"양", "신":"음", "임":"양", "계":"음"
}
branch_element_map = {
    "자":"수", "축":"토", "인":"목", "묘":"목", "진":"토", "사":"화",
    "오":"화", "미":"토", "신":"금", "유":"금", "술":"토", "해":"수"
}
branch_yinyang_map = {
    "자":"음", "축":"음", "인":"양", "묘":"음", "진":"양", "사":"양",
    "오":"음", "미":"음", "신":"양", "유":"음", "술":"양", "해":"양"
}

# 절기 → 월지 매핑
SOLAR_TERMS = [(1,6), (2,4), (3,6), (4,5), (5,6), (6,6),
    (7,7), (8,8), (9,8), (10,8), (11,7), (12,7)]
MONTH_BRANCH = ["축","인","묘","진","사","오","미","신","유","술","해","자"]
MONTH_START_GAN = {
    "갑":"병", "기":"병", "을":"무", "경":"무", "병":"경", "신":"경",
    "정":"임", "임":"임", "무":"갑", "계":"갑"
}
HOUR_START_GAN = {
    "갑":"갑", "을":"병", "병":"무", "정":"경", "무":"임",
    "기":"갑", "경":"병", "신":"무", "임":"경", "계":"임"
}

# ======================
# 명반 계산기
# ======================
def get_year_ganji(y, m, d):
    ipchun = datetime(y,2,4)
    base_y = y if datetime(y,m,d) >= ipchun else y - 1
    return sexagenary[(base_y - 4) % 60]

def month_index_by_terms(y, m, d):
    dt = datetime(y,m,d)
    idx = None
    for i, (mm,dd) in enumerate(SOLAR_TERMS):
        if dt >= datetime(y,mm,dd):
            idx = i
    return 11 if idx is None else idx

def get_month_ganji(y, m, d):
    yg = get_year_ganji(y,m,d)[0]
    mi_raw = month_index_by_terms(y,m,d)
    mi = (mi_raw - 1) % 12
    branch = MONTH_BRANCH[mi_raw]
    start = MONTH_START_GAN[yg]
    stem = stems[(stems.index(start) + mi) % 10]
    return stem + branch

def get_day_ganji(y, m, d):
    base = datetime(1900,1,31)
    delta = (datetime(y,m,d) - base).days
    return sexagenary[(delta + 40) % 60]

def zhi_index(hh, mm):
    t = hh * 60 + mm
    if mm == 0 and hh % 2 == 1:
        t -= 1
    k = (t - 23*60) % (24*60)
    return k // 120

def get_hour_ganji(day_gan, hh, mm):
    idx = zhi_index(hh, mm)
    branch = branches[idx]
    start = HOUR_START_GAN[day_gan]
    stem = stems[(stems.index(start) + idx) % 10]
    return stem + branch

def get_myeongpan(y, m, d, hh, mm, gender="여"):
    ny = get_year_ganji(y,m,d)
    nm = get_month_ganji(y,m,d)
    nd = get_day_ganji(y,m,d)
    nh = get_hour_ganji(nd[0], hh, mm)
    return {
        "성별": gender,
        "년주": [ny[0], ny[1]],
        "월주": [nm[0], nm[1]],
        "일주": [nd[0], nd[1]],
        "시주": [nh[0], nh[1]]
    }

# ======================
# 십성
# ======================
def get_sipseong(ilgan, target, is_gan=True):
    my_elem = element_map[ilgan]
    my_yy   = yin_yang_map[ilgan]

    if is_gan:
        tg_elem = element_map.get(target)
        tg_yy = yin_yang_map.get(target)
    else:
        tg_elem = branch_element_map.get(target)
        tg_yy = branch_yinyang_map.get(target)

    if not tg_elem or not tg_yy:
        return "?"

    if my_elem == tg_elem:
        return "비견" if my_yy == tg_yy else "겁재"

    order = ["목", "화", "토", "금", "수"]
    diff = (order.index(tg_elem) - order.index(my_elem)) % 5
    if diff == 1: return "식신" if my_yy == tg_yy else "상관"
    if diff == 2: return "편재" if my_yy == tg_yy else "정재"
    if diff == 3: return "편관" if my_yy == tg_yy else "정관"
    if diff == 4: return "편인" if my_yy == tg_yy else "정인"
    return "-"

# ======================
# 십이운성
# ======================
gan_group_map = {
    "갑": "갑", "을": "을",
    "병": "병/무", "무": "병/무",
    "정": "정/기", "기": "정/기",
    "경": "경", "신": "신",
    "임": "임", "계": "계"
}
sibyung_table = {
    "자": {"갑": "목욕", "을": "병", "병/무": "태", "정/기": "절", "경": "사", "신": "장생", "임": "제왕", "계": "건록"},
    "축": {"갑": "관대", "을": "쇠", "병/무": "양", "정/기": "묘", "경": "묘", "신": "제왕", "임": "건록", "계": "장생"},
    "인": {"갑": "건록", "을": "제왕", "병/무": "장생", "정/기": "사", "경": "절", "신": "태", "임": "병", "계": "목욕"},
    "묘": {"갑": "제왕", "을": "건록", "병/무": "목욕", "정/기": "병", "경": "태", "신": "절", "임": "사", "계": "장생"},
    "진": {"갑": "쇠", "을": "관대", "병/무": "관대", "정/기": "쇠", "경": "양", "신": "묘", "임": "묘", "계": "양"},
    "사": {"갑": "병", "을": "목욕", "병/무": "건록", "정/기": "제왕", "경": "장생", "신": "사", "임": "절", "계": "태"},
    "오": {"갑": "사", "을": "장생", "병/무": "제왕", "정/기": "건록", "경": "목욕", "신": "병", "임": "태", "계": "절"},
    "미": {"갑": "묘", "을": "양", "병/무": "쇠", "정/기": "관대", "경": "관대", "신": "쇠", "임": "양", "계": "묘"},
    "신": {"갑": "절", "을": "태", "병/무": "병", "정/기": "목욕", "경": "건록", "신": "제왕", "임": "장생", "계": "사"},
    "유": {"갑": "태", "을": "절", "병/무": "사", "정/기": "장생", "경": "제왕", "신": "건록", "임": "목욕", "계": "병"},
    "술": {"갑": "양", "을": "묘", "병/무": "묘", "정/기": "양", "경": "쇠", "신": "관대", "임": "관대", "계": "쇠"},
    "해": {"갑": "장생", "을": "사", "병/무": "절", "정/기": "태", "경": "병", "신": "목욕", "임": "건록", "계": "제왕"}
}
def get_sibyun(ilgan, branch):
    gan_group = gan_group_map.get(ilgan)
    if not gan_group or branch not in sibyung_table:
        return "?"
    return sibyung_table[branch].get(gan_group, "?")

# ======================
# 십이신살
# ======================
SHINSAL_NAMES = ["겁살","재살","천살","지살","연살","월살","망신살","장성살","반안살","역마살","육해살","화개살"]
SAMHAP_LINES = {
    "해묘미": ["해","묘","미"],
    "인오술": ["인","오","술"],
    "사유축": ["사","유","축"],
    "신자진": ["신","자","진"],
}
LINE_INFO = {
    "해묘미":{"국":"목국","순서":["신","유","술","해","자","축","인","묘","진","사","오","미"]},
    "인오술":{"국":"화국","순서":["해","자","축","인","묘","진","사","오","미","신","유","술"]},
    "사유축":{"국":"금국","순서":["인","묘","진","사","오","미","신","유","술","해","자","축"]},
    "신자진":{"국":"수국","순서":["사","오","미","신","유","술","해","자","축","인","묘","진"]},
}
def find_line(standard_branch):
    for line_name, members in SAMHAP_LINES.items():
        if standard_branch in members:
            guk = LINE_INFO[line_name]["국"]
            return line_name, guk
    return None, None
def make_line_mapping(line_name):
    order = LINE_INFO[line_name]["순서"]
    return dict(zip(order, SHINSAL_NAMES))
def get_shinsal(standard_branch, target_branch):
    line_name, _ = find_line(standard_branch)
    if not line_name: return "-"
    mapping = make_line_mapping(line_name)
    return mapping.get(target_branch, "-")

# ======================
# 종합 DataFrame 생성
# ======================
def build_myeongpan_df(y, m, d, hh, mm, gender="여"):
    mp = get_myeongpan(y, m, d, hh, mm, gender)
    ilgan = mp["일주"][0]

    rows = []
    for label in ["년주","월주","일주","시주"]:
        gan, ji = mp[label]

        gan_sip = get_sipseong(ilgan, gan, is_gan=True)
        ji_sip  = get_sipseong(ilgan, ji, is_gan=False)

        # 🔹 일간은 (나)로 표시
        if label == "일주":
            gan_disp = f"{gan}{element_map[gan]}(본원)"
        else:
            gan_disp = f"{gan}{element_map[gan]}({gan_sip})"

        ji_disp  = f"{ji}{branch_element_map[ji]}({ji_sip})"

        sibyun_val = get_sibyun(ilgan, ji)
        shinsal_year = get_shinsal(mp["년주"][1], ji)
        shinsal_day  = get_shinsal(mp["일주"][1], ji)

        rows.append({
            "지점": label,
            "천간(십성)": gan_disp,
            "지지(십성)": ji_disp,
            "십이운성": sibyun_val,
            "십이신살(년주기준)": shinsal_year,
            "십이신살(일주기준)": shinsal_day
        })

    df = pd.DataFrame(rows)
    df.attrs["제목"] = f"출생: {gender} / {y}년 {m}월 {d}일 {hh}시 {mm}분"
    return mp, df

# ======================
# 오행분포 / 십성분포
# ======================
def _five_group_map_by_day(day_elem):
    order = ["목", "화", "토", "금", "수"]
    i = order.index(day_elem)
    return {
        order[i]: "비겁",
        order[(i+1) % 5]: "식상",
        order[(i+2) % 5]: "재성",
        order[(i+3) % 5]: "관성",
        order[(i+4) % 5]: "인성",
    }

def summarize_element_distribution(myeongpan):
    ilgan = myeongpan["일주"][0]
    day_elem = element_map[ilgan]
    elem_to_group = _five_group_map_by_day(day_elem)

    elem_counter = Counter()
    for label in ["년주","월주","일주","시주"]:
        gan, ji = myeongpan[label]
        elem_counter[element_map[gan]] += 1
        elem_counter[branch_element_map[ji]] += 1

    return elem_counter, elem_to_group

def summarize_tengod_distribution(myeongpan):
    ilgan = myeongpan["일주"][0]
    sip_counter = Counter()
    for label in ["년주","월주","일주","시주"]:
        gan, ji = myeongpan[label]

        # 🔹 일간은 십성 계산에서 제외
        if not (label == "일주" and gan == ilgan):
            gan_sip = get_sipseong(ilgan, gan, is_gan=True)
            sip_counter[gan_sip] += 1

        ji_sip = get_sipseong(ilgan, ji, is_gan=False)
        sip_counter[ji_sip]  += 1

    return sip_counter

# ======================
# 신강/신약 및 용희신/기구신 판별
# ======================
GAN_WEIGHTS = {"시간":5, "일간":10, "월간":10, "년간":5}
JI_WEIGHTS  = {"시지":10, "일지":15, "월지":35, "년지":10}

def judge_useful_gods(myeongpan):
    elem_counts = {e:0 for e in ["목","화","토","금","수"]}
    for label in ["년주","월주","일주","시주"]:
        gan, ji = myeongpan[label]
        elem_counts[element_map[gan]] += 1
        elem_counts[branch_element_map[ji]] += 1

    min_val = min(elem_counts.values())
    max_val = max(elem_counts.values())

    min_elems = [e for e,v in elem_counts.items() if v == min_val]
    max_elems = [e for e,v in elem_counts.items() if v == max_val]

    return {"용희신오행": min_elems, "기구신오행": max_elems}


def judge_yongshin_simple(myeongpan):
    ilgan = myeongpan["일주"][0]
    total_score = 0
    in_score, bi_score = 0, 0
    jae_score, gwan_score, sik_score = 0, 0, 0

    plus_group = {"비견":"비겁","겁재":"비겁","편인":"인성","정인":"인성"}
    minus_group = {
        "식신":"식상","상관":"식상",
        "편재":"재성","정재":"재성",
        "편관":"관성","정관":"관성"
    }

    for label in ["년주","월주","일주","시주"]:
        gan, ji = myeongpan[label]

        # 천간
        gan_sip = get_sipseong(ilgan, gan, is_gan=True)
        g_weight = GAN_WEIGHTS[label.replace("주","간")]
        if gan_sip in plus_group:
            total_score += g_weight
            if plus_group[gan_sip] == "인성": in_score += g_weight
            if plus_group[gan_sip] == "비겁": bi_score += g_weight
        elif gan_sip in minus_group:
            total_score -= g_weight
            if minus_group[gan_sip] == "재성": jae_score += g_weight
            if minus_group[gan_sip] == "관성": gwan_score += g_weight
            if minus_group[gan_sip] == "식상": sik_score += g_weight

        # 지지
        ji_sip = get_sipseong(ilgan, ji, is_gan=False)
        j_weight = JI_WEIGHTS[label.replace("주","지")]
        if ji_sip in plus_group:
            total_score += j_weight
            if plus_group[ji_sip] == "인성": in_score += j_weight
            if plus_group[ji_sip] == "비겁": bi_score += j_weight
        elif ji_sip in minus_group:
            total_score -= j_weight
            if minus_group[ji_sip] == "재성": jae_score += j_weight
            if minus_group[ji_sip] == "관성": gwan_score += j_weight
            if minus_group[ji_sip] == "식상": sik_score += j_weight

    # ======================
    # 판별 규칙
    # ======================
    if total_score >= 50:
        state = "신강"
        if in_score > bi_score:
            yong, kigu = "식재", "관인"   # 신강 + 인성 > 비겁
        else:
            yong, kigu = "재관", "인비"   # 신강 + 비겁 > 인성
    elif total_score <= -50:
        state = "신약"
        yong, kigu = "인비", "식재관"     # 신약 고정
    else:
        state = "중화"
        yg = judge_useful_gods(myeongpan)
        yong, kigu = yg['용희신'], yg['기구신']
        min_elem, max_elem = yg['용희신'], yg['기구신']

    yg = judge_useful_gods(myeongpan)

    return {
        "점수": total_score,
        "판정": state,
        "인성점수": in_score,
        "비겁점수": bi_score,
        "재성점수": jae_score,
        "관성점수": gwan_score,
        "식상점수": sik_score,
        "용희신": yong,
        "기구신": kigu,
        "용희신오행": yg["용희신오행"],
        "기구신오행": yg["기구신오행"]
    }


def judge_gegguk_10(myeongpan):
    """
    월지(月支)의 십성으로 격국 이름과 성격을 상세(10개)로 반환
    반환 예: {"월지십성":"편관", "격국":"편관격", "성격":"...설명..."}
    """
    ilgan = myeongpan["일주"][0]
    _, wolji = myeongpan["월주"]

    wolji_sip = get_sipseong(ilgan, wolji, is_gan=False)
    gegguk = GEGGUK_LABEL_BY_SIPSUNG.get(wolji_sip, "기타격")
    trait = GEGGUK_TRAITS_10.get(wolji_sip, "-")

    return {"월지십성": wolji_sip, "격국": gegguk, "성격": trait}


# ======================
# 기질 요약 데이터베이스
# ======================

# 오행 기질
OHAENG_TRAITS = {
    "목": {
        "과다": "고집이 강하고 자기 주장 강함, 성급하고 공격적이 될 수 있음. 계획은 크지만 실천이 부족할 수 있음.",
        "적당": "진취적이고 성장 지향적, 도전정신이 있으며 배려심이 있음. 인간관계에서 따뜻함.",
        "부족": "의욕이 부족하고 추진력이 약함, 결단력이 부족해 우유부단할 수 있음. 자기표현에 소극적."
    },
    "화": {
        "과다": "성격이 급하고 예민하며 감정 기복이 심함. 지나친 자존심, 분노 조절 문제 가능.",
        "적당": "활발하고 열정적, 밝고 적극적인 성격. 리더십과 추진력이 좋음.",
        "부족": "활력이 부족하고 소극적이며 기회 포착이 늦음. 의욕과 열정이 잘 드러나지 않음."
    },
    "토": {
        "과다": "지나치게 신중하고 답답함. 고집스럽고 융통성이 부족할 수 있음. 소유욕·집착 심화.",
        "적당": "성실하고 책임감 강함, 안정적이고 신뢰할 수 있는 성격. 중재와 조화 능력 뛰어남.",
        "부족": "중심이 부족해 흔들림이 많고, 책임감 부족. 안정감·균형감이 약해 불안정한 모습."
    },
    "금": {
        "과다": "지나치게 냉정하고 공격적, 비판적 태도. 지나친 경쟁심과 자기중심성.",
        "적당": "결단력이 있고 원칙을 지키며 정의로움. 의리와 신뢰를 중요시.",
        "부족": "결단력 부족, 소심하고 우유부단. 자기 방어가 약해 타인에게 휘둘림."
    },
    "수": {
        "과다": "지나치게 계산적, 의심 많음. 현실 회피 성향, 불안정한 마음. 감정 기복 심함.",
        "적당": "지혜롭고 사려 깊음. 융통성이 있고 상황에 적응 잘함. 사고가 깊고 창의력 풍부.",
        "부족": "사고의 깊이 부족, 융통성 결여. 변화를 두려워하며 창의력 발휘가 어려움."
    }
}

# 십성 그룹 기질
SIPSUNG_TRAITS = {
    "비겁": {
        "과다": "자기주장이 강하고 독립심이 지나쳐 협력이 부족. 고집이 세고 경쟁심이 과도함.",
        "적당": "자신감과 리더십이 있으며 의리 있고 협동심이 좋음. 인간관계에서 든든한 버팀목.",
        "부족": "자기주장이 약하고 의존적. 소극적이고 주도성이 부족."
    },
    "인성": {
        "과다": "생각이 너무 많고 현실성이 떨어짐. 의존적이고 게으름 가능성. 공부·이론에 치중.",
        "적당": "학문적, 직관적, 지혜롭고 타인을 이해하는 힘. 배움과 조언자 역할.",
        "부족": "학습 능력·사고력 부족. 도움받기 어려우며 자기 믿음이 약해 불안정."
    },
    "재성": {
        "과다": "물질적 집착, 소비 과다. 이성관계 방탕 가능. 욕심이 지나쳐 불안정.",
        "적당": "현실적, 경제적 능력 좋음. 책임감과 성실함, 가정과 재물 관리에 강점.",
        "부족": "경제적 감각 부족, 재물 관리 미숙. 책임감 부족, 현실에 약함."
    },
    "관성": {
        "과다": "권위적, 고압적 태도. 책임에 짓눌림, 스트레스 과다. 억압적 성격.",
        "적당": "법과 원칙을 중시, 책임감·도덕성 강함. 사회적 신뢰와 성취를 추구.",
        "부족": "원칙·책임감 부족, 게으르고 타인 규율에 무관심. 목표식 추진력 약함."
    },
    "식상": {
        "과다": "말이 많고 산만함. 자기표현이 지나쳐 오만하거나 반항적. 예술적 재능은 크나 불안정.",
        "적당": "창의력, 표현력, 예술적 감각 뛰어남. 활동적이고 재능 발휘.",
        "부족": "표현력이 부족하고 자기재능 발휘 어려움. 창의력·사교성 약화."
    }
}

# 신강/신약 기질
SHINKANG_TRAITS = {
    "신강": "자기주장이 강하고 추진력 있음. 결단력과 주도성이 크지만 독선적일 수 있음.",
    "신약": "소극적이고 외부 영향에 민감. 유연하고 조화를 잘 이루지만 추진력이 부족함."
}

# 용희신/기구신 (오행 관점)
YONGGI_TRAITS = {
    "용희신": {
        "목": "진취적, 개척정신, 인맥 도움.",
        "화": "열정과 추진력, 명예·성과.",
        "토": "안정, 책임감, 신뢰 구축.",
        "금": "결단력, 정의, 원칙성.",
        "수": "지혜, 융통성, 창의성."
    },
    "기구신": {
        "목": "고집, 과도한 경쟁심, 충돌.",
        "화": "분노, 예민함, 인간관계 갈등.",
        "토": "집착, 융통성 부족, 답답함.",
        "금": "지나친 냉정함, 갈등, 비판적 성향.",
        "수": "의심, 불안정, 감정 기복."
    }
}

# 용희신/기구신 조합 → 십성 그룹 분해 매핑
COMBO_MAP = {
    "재관": ["재성", "관성"],
    "인비": ["인성", "비겁"],
    "식재": ["식상", "재성"],
    "식관": ["식상", "관성"],
    "인식": ["인성", "식상"],
    "재인": ["재성", "인성"],
    "식재관": ["식상", "재성", "관성"]   # 🔹 추가
}


# 십이신살 기질
SHINSAL_TRAITS = {
    "장성살": "지도력, 명예, 권위. 리더십 강점.",
    "망신살": "예술·연예·창의적 표현 재능, 매력 발산.",
    "반안살": "개혁 정신, 새로운 전환 능력, 창업·혁신.",
    "화개살": "종교·철학·예술적 탐구, 정신세계의 깊이.",
    "도화살": "인기, 대인관계 매력, 예술적 감각.",
    "월살": "독창적 아이디어, 변화 감각, 불안정성도 동반.",
    "겁살": "강한 행동력, 도전정신, 위험 감수.",
    "재살": "비범한 기획력, 파괴 후 재건 능력.",
    "천살": "비범한 영감, 예술·학문적 재능.",
    "지살": "여행, 이동, 새로운 환경 적응력.",
    "역마살": "활동적, 이동·여행 많음, 변화에 강함.",
    "육해살": "사회적 관계·협력 능력, 대인관계 다양성."
}

# 용희신/기구신 (십성 관점)
YONGGI_SIPSUNG_TRAITS = {
    "비겁": {
        "용희신": "강한 추진력, 독립심, 도전 정신. 리더십과 협동을 동시에 발휘. 어려움 극복의 원동력.",
        "기구신": "지나친 경쟁심, 독선, 고집. 타인과 갈등이 많고 협력 부재로 문제 발생."
    },
    "인성": {
        "용희신": "지혜와 학문, 통찰력 발휘. 보호와 도움을 받고, 학업·연구·전문 분야에서 성취.",
        "기구신": "과도한 의존, 현실 회피, 게으름. 타인의 도움에 지나치게 기대며 소극적이 됨."
    },
    "재성": {
        "용희신": "재물·현실 감각 뛰어남. 경제적 성취와 안정, 책임감 강화. 가족·대인 관계에서 신뢰 얻음.",
        "기구신": "욕심·집착 심해짐. 재물 관리 실패, 지나친 소비. 이성 문제·유흥에 빠지기 쉬움."
    },
    "관성": {
        "용희신": "책임감, 도덕성, 사회적 성취. 리더십과 권위가 긍정적으로 작용. 조직에서 인정받음.",
        "기구신": "억압, 스트레스, 권위 남용. 책임에 짓눌리고 심리적 불안정. 대인 갈등 증가."
    },
    "식상": {
        "용희신": "창의력, 자기표현, 예술·언변 재능 발휘. 활동적이고 생산적 에너지.",
        "기구신": "산만하고 반항적, 말이 지나쳐 갈등 발생. 표현력은 많으나 방향성 잃고 소모적."
    }
}

# 개별 십성 기질
INDIVIDUAL_SIPSUNG_TRAITS = {
    "비견": "자신감 강하고 독립적, 친구·동료와 협력 잘함. 자존심 강하고 주도적.",
    "겁재": "경쟁심 강하고 모험적, 추진력 있으나 다소 충동적. 재물 기복이 심하고 의리 강함.",
    "정인": "따뜻하고 포용적. 학문적, 지혜롭고 남을 잘 돕는 성향. 모성·부성적 기질.",
    "편인": "창의적, 예술적 감각, 독창성 뛰어남. 현실적응이 약하고 변덕스러울 수 있음. 예술가·철학가 기질.",
    "정재": "성실, 근면, 책임감 강함. 가정적이고 꾸준히 재물 축적. 안정 지향.",
    "편재": "사교적, 활동적, 대범함. 기회 포착 능력 뛰어남. 돈·이성 욕구 강하고 변화 많음.",
    "정관": "책임감, 도덕성, 원칙·규율 중시. 공직자적 성향, 신뢰와 권위 추구.",
    "편관": "결단력, 강한 추진력, 카리스마. 급하고 공격적일 수 있으나 위기 대처 능력 뛰어남.",
    "식신": "여유롭고 낙천적, 생산적·창조적 성향. 베푸는 성격, 예술·창작 재능.",
    "상관": "말 잘하고 표현력 뛰어남. 창의적이나 반항적·비판적. 권위와 충돌하기 쉬움."
}

# 십성 → 5그룹 매핑 (추가)
SIPS_TO_GROUP = {
    "비견": "비겁", "겁재": "비겁",
    "식신": "식상", "상관": "식상",
    "편재": "재성", "정재": "재성",
    "편관": "관성", "정관": "관성",
    "편인": "인성", "정인": "인성",
}

# ======================
# 격국 판별 (월지 십성 기준, 10개 구분)
# ======================

GEGGUK_LABEL_BY_SIPSUNG = {
    "비견": "건록격",
    "겁재": "양인격",
    "식신": "식신격",
    "상관": "상관격",
    "정재": "정재격",
    "편재": "편재격",
    "정관": "정관격",
    "편관": "편관격",
    "정인": "정인격",
    "편인": "편인격",
}

GEGGUK_TRAITS_10 = {
    "비견": "자기주장 강하고 독립심·주도성 뚜렷. 동료와 협력도 가능하나 과하면 고집·충돌.",
    "겁재": "공격적 추진력과 승부욕, 빠른 결단. 과하면 무리수·권위와 충돌·리스크 과다.",
    "식신": "생산·창조·실무력, 꾸준함과 후덕함. 과하면 안일·느슨·속도 저하.",
    "상관": "표현·비판·혁신, 창의·기획 재능. 과하면 권위도전·말로 갈등·산만.",
    "정재": "현실감각·저축·계획성, 가정·조직 안정과 신뢰. 과하면 집착·안정지향 과도.",
    "편재": "외향·대범·네트워크·영업/사업 감각. 과하면 소비·변덕·대인관계 소모.",
    "정관": "원칙·법·책임, 공직·관리형 역량과 신뢰. 과하면 경직·형식주의.",
    "편관": "결단·위기대응·카리스마, 강한 리더십. 과하면 공격성·스트레스·압박.",
    "정인": "학문·보호·배려·전문성, 내공과 신뢰. 과하면 의존·이론과잉·현실감 저하.",
    "편인": "직관·창의·독자성, 예술·철학 성향. 과하면 현실도피·변덕·실행력 저하.",
}

# ======================
# 지지(동물) 성향 / 직업 힌트
# ======================
BRANCH_ANIMAL_TRAITS = {
    "자": ("쥐", "민첩·실용·정보감각. 기회 포착/네트워킹 강점. 과하면 계산·예민으로 관계 피로."),
    "축": ("소", "끈기·신뢰·현실감각. 규율·품질관리 강점. 과하면 완고·의사결정 지연."),
    "인": ("호랑이", "개척·용기·리더십. 위기 돌파/추진형. 과하면 독선·충돌·세부 미흡."),
    "묘": ("토끼", "온화·조화·미감. 관계 중재·교육·상담/디자인 강점. 과하면 우유부단."),
    "진": ("용", "비전·상징·브랜딩. 영향력 확장/스토리텔링. 과하면 과신·실행력 저하."),
    "사": ("뱀", "분석·통찰·전략. 연구/리스크관리/설계 강점. 과하면 비밀주의·의심."),
    "오": ("말", "활동·낙관·사교. 현장/홍보/영업 강점. 과하면 변덕·마무리 약화."),
    "미": ("양", "배려·공동체·예술감성. 케어/문화/콘텐츠 품질. 과하면 감정기복·결정 지연."),
    "신": ("원숭이", "재치·학습속도·툴 습득. 문제해결/크로스플레이. 과하면 산만·일관성 저하."),
    "유": ("닭", "정확·규범·디테일. 품질/감사/문서화 강점. 과하면 완벽주의·비판 과다."),
    "술": ("개", "충성·정의·수호. 리스크 통제/안전/신뢰. 과하면 흑백논리·고집."),
    "해": ("돼지", "포용·복덕·회복탄력. 복지/서비스/파트너십. 과하면 안일·경계심 부족."),
}

# 지지(동물) → 직업 클러스터 힌트(가중치)
BRANCH_JOB_HINTS = {
    "자": {"DATA_SCI":1.2, "CSM":1.0, "SECURITY":0.8, "BIZ_ANALYST":0.8},
    "축": {"FIN_ACC":1.2, "QAQC_MFG":1.0, "OPS":1.0, "PUBLIC":0.6},
    "인": {"PM_PRODUCT":1.2, "STRATEGY":1.0, "CONSULT":1.0, "SALES_B2B":0.8},
    "묘": {"UX":1.2, "EDU":1.0, "COUNSEL":1.0, "UI_VISUAL":0.8, "BRAND_PR":0.6},
    "진": {"BRAND_PR":1.0, "PM_PRODUCT":0.8, "PUBLIC":0.8, "RND":0.6},
    "사": {"SECURITY":1.0, "LAW":0.8, "DATA_SCI":0.8, "RND":0.8, "STRATEGY":0.6},
    "오": {"BRAND_PR":1.0, "SALES_B2B":1.0, "CONTENT":0.8, "PM_PRODUCT":0.6},
    "미": {"CONTENT":1.0, "UX":0.8, "EDU":0.8, "CSM":0.6, "BRAND_PR":0.6},
    "신": {"BACKEND":1.0, "FRONTEND":0.8, "MOBILE":0.8, "QAQC_SW":0.6, "DEVOPS":0.6},
    "유": {"QAQC_SW":1.0, "LAW":0.8, "FIN_ACC":0.8, "HSE":0.6},
    "술": {"HSE":1.0, "PUBLIC":0.8, "SECURITY":0.8, "OPS":0.6},
    "해": {"COUNSEL":1.0, "EDU":0.8, "PUBLIC":0.6, "AM_PM":0.6},
}
# 일지 가중치(본인) vs 기타 지지 가중치
DAY_BRANCH_WEIGHT = 1.1
OTHER_BRANCH_WEIGHT = 0.35

# ======================
# 직업군 추천: 도메인/매핑/스코어러
# ======================

# 1) 직업군 클러스터 (다양·구체)
CLUSTERS = {
    # Tech · Data
    "DATA_SCI":{"name":"데이터사이언스","tags":["분석","수리","통계"]},
    "ML_ENG":{"name":"머신러닝/AI엔지니어","tags":["ML","배포","MLOps"]},
    "DS_ENG":{"name":"데이터엔지니어","tags":["데이터플랫폼","ETL","파이프라인"]},
    "BACKEND":{"name":"백엔드 개발","tags":["서버","API","DB"]},
    "FRONTEND":{"name":"프론트엔드 개발","tags":["웹","UI","JS"]},
    "MOBILE":{"name":"모바일 앱 개발","tags":["iOS","Android","앱"]},
    "SECURITY":{"name":"정보보안","tags":["보안","리스크","규정"]},
    "QAQC_SW":{"name":"소프트웨어 QA","tags":["테스트","품질","검증"]},
    "DEVOPS":{"name":"DevOps/SRE","tags":["인프라","자동화","신뢰성"]},

    # Strategy · Biz
    "PM_PRODUCT":{"name":"프로덕트 매니저","tags":["기획","조율","우선순위"]},
    "STRATEGY":{"name":"전략/경영기획","tags":["분석","의사결정","지표"]},
    "CONSULT":{"name":"경영 컨설팅","tags":["문제해결","리서치","고객"]},
    "BIZ_ANALYST":{"name":"비즈니스 분석가","tags":["지표","대시보드","의사결정"]},
    "FIN_ACC":{"name":"회계/세무","tags":["정밀","규율","보고"]},
    "FIN_IB":{"name":"투자/IB/딜","tags":["성과","딜","속도"]},
    "AM_PM":{"name":"자산운용/포트폴리오","tags":["분석","리스크","금융"]},
    "SALES_B2B":{"name":"B2B 영업","tags":["대인","성과","관계"]},
    "CSM":{"name":"고객성공/CSM","tags":["관계","지속","문제해결"]},

    # Public · Law
    "LAW":{"name":"법무/컴플라이언스","tags":["규범","계약","리스크"]},
    "PUBLIC":{"name":"공공/정책/행정","tags":["공공성","제도","안정"]},
    "EDU":{"name":"교육/강의/HRD","tags":["지식전달","커리큘럼","코칭"]},
    "COUNSEL":{"name":"심리상담/코칭","tags":["공감","치유","경청"]},

    # Operation · Supply
    "OPS":{"name":"운영/오퍼레이션","tags":["프로세스","안정","실무"]},
    "SCM":{"name":"물류/SCM","tags":["공급망","수급","현장"]},
    "QAQC_MFG":{"name":"제조 QA/QC","tags":["품질","표준","검사"]},
    "HSE":{"name":"안전/보건/HSE","tags":["안전","리스크","준수"]},
    "PM_CONS":{"name":"건설/현장 PM","tags":["현장","리스크","조율"]},

    # Design · Content
    "UX":{"name":"UX 리서치/디자인","tags":["사용자","리서치","프로토타입"]},
    "UI_VISUAL":{"name":"UI/비주얼 디자인","tags":["시각","브랜딩","가이드"]},
    "GAME_DESIGN":{"name":"게임 기획/레벨","tags":["창의","시스템","콘텐츠"]},
    "CONTENT":{"name":"콘텐츠 기획/제작","tags":["스토리","표현","미디어"]},
    "BRAND_PR":{"name":"브랜드/PR/커뮤니케이션","tags":["메시지","미디어","전략"]},

    # Science · R&D
    "RND":{"name":"R&D/연구","tags":["탐구","실험","논문"]},
    "BIO_HEALTH":{"name":"바이오/헬스케어","tags":["임상","연구","규정"]},
    "EARTH_ENV":{"name":"환경/지구과학","tags":["환경","지속가능","분석"]},
}

# ======================
# 전공(학과) 카탈로그 & 매핑
# ======================

# 전공 카탈로그 (key -> 표시명)
MAJOR_CATALOG = {
    # 공학/이과
    "MECH":"기계공학", "EE":"전기공학", "ECE":"전자공학", "SEMIS":"반도체공학",
    "IND":"산업공학", "MSE":"재료/신소재", "CIV":"토목공학", "CHEME":"화학공학",
    "CS":"컴퓨터공학", "SW":"소프트웨어", "SEC":"정보보안", "AERO":"항공/우주",
    "ENVENG":"환경공학", "ARCH":"건축학", "BIO":"생명과학/바이오", "STATS":"통계학",
    "MATH":"수학", "PHYS":"물리학", "EARTH":"지구과학",
    # 경영/법/사회
    "ACC":"회계학", "FIN":"금융/재무", "LAW":"법학", "PA":"행정학",
    "ECON":"경제학", "PSY":"심리학", "SOC":"사회학", "EDU":"교육학",
    "COMM":"커뮤니케이션/언론", "MKT":"마케팅", "BUS":"경영학",
    # 인문/철학
    "PHIL":"철학", "HIST":"역사학", "LANG":"언어/문학",
    # 디자인/예체능
    "UX":"UX/HCI", "VISUAL":"시각/그래픽디자인", "IND_DES":"산업디자인",
    "MUSIC":"음악", "FINEART":"미술/순수예술", "THEATRE":"연극/영화",
    "PE":"체육/스포츠과학",
}

# 직업군 → 어울리는 전공(기본 연결)
JOB_TO_MAJORS = {
    # Tech · Data
    "DATA_SCI":["STATS","MATH","CS","ECON","IND"],
    "ML_ENG":["CS","MATH","STATS","ECE"],
    "DS_ENG":["CS","SW","IND","MATH"],
    "BACKEND":["CS","SW","ECE"],
    "FRONTEND":["CS","SW","UX"],
    "MOBILE":["CS","SW","ECE"],
    "SECURITY":["SEC","CS","LAW","ECE"],
    "QAQC_SW":["CS","SW","IND"],
    "DEVOPS":["CS","SW","IND","EE"],

    # Strategy · Biz
    "PM_PRODUCT":["CS","UX","BUS","IND","MKT"],
    "STRATEGY":["BUS","ECON","IND","MATH"],
    "CONSULT":["BUS","ECON","IND","STATS"],
    "BIZ_ANALYST":["STATS","MATH","BUS","ECON"],
    "FIN_ACC":["ACC","FIN","ECON","MATH"],
    "FIN_IB":["FIN","ECON","MATH"],
    "AM_PM":["FIN","ECON","MATH","IND"],
    "SALES_B2B":["BUS","MKT","IND_DES"],
    "CSM":["BUS","COMM","PSY"],

    # Public · Law
    "LAW":["LAW","POL" if "POL" in MAJOR_CATALOG else "PA"],  # POL 키가 없으면 PA로 대체
    "PUBLIC":["PA","LAW","ECON"],
    "EDU":["EDU","PSY","PHIL"],
    "COUNSEL":["PSY","SOC","EDU","PHIL"],

    # Operation · Supply
    "OPS":["IND","BUS","ACC"],
    "SCM":["IND","BUS","ECON"],
    "QAQC_MFG":["IND","MSE","MECH"],
    "HSE":["ENVENG","PA","LAW"],
    "PM_CONS":["CIV","ARCH","IND"],

    # Design · Content
    "UX":["UX","HCI" if "HCI" in MAJOR_CATALOG else "CS","PSY"],
    "UI_VISUAL":["VISUAL","IND_DES"],
    "GAME_DESIGN":["CS","SW","UX","VISUAL"],
    "CONTENT":["COMM","MKT","LANG","HIST"],
    "BRAND_PR":["COMM","MKT","VISUAL"],

    # Science · R&D
    "RND":["PHYS","CHEME","MSE","BIO","MATH","STATS"],
    "BIO_HEALTH":["BIO","CHEME","MSE"],
    "EARTH_ENV":["EARTH","ENVENG","STATS"],
}

# 오행 → 전공 힌트(금=엔지니어링 강화, 철학/예체능 포함)
MAP_ELEMENT_TO_MAJORS = {
    "금": ["MECH","EE","ECE","SEMIS","IND","MSE","AERO","SEC","LAW"],
    "토": ["CIV","ARCH","ACC","PA","ENVENG","IND"],
    "수": ["CS","SW","STATS","MATH","PHIL","ECON","PSY"],
    "목": ["EDU","PSY","SOC","UX","COMM","ENVENG","EARTH"],
    "화": ["MKT","COMM","THEATRE","MUSIC","FINEART","PE","CONTENT"],
}

# 2) 오행 → 직업군 가중치
MAP_BY_ELEMENT = {
    "목": {"PM_PRODUCT":3,"UX":2,"EDU":2,"STRATEGY":1,"BRAND_PR":1},
    "화": {"SALES_B2B":3,"BRAND_PR":2,"PM_PRODUCT":1,"CONTENT":1},
    "토": {"OPS":3,"SCM":2,"FIN_ACC":1,"PUBLIC":1,"QAQC_MFG":1},
    "금": {"LAW":3,"SECURITY":2,"QAQC_SW":2,"FIN_ACC":1,"HSE":1},
    "수": {"DATA_SCI":3,"ML_ENG":2,"DS_ENG":2,"COUNSEL":1,"BACKEND":1,"RND":1},
}

# 3) 십성(10개) → 직업군 가중치
MAP_BY_TENGOD_10 = {
    "비견":{"PM_PRODUCT":2,"SECURITY":1,"BACKEND":1},
    "겁재":{"SALES_B2B":3,"SECURITY":2,"OPS":1},
    "식신":{"OPS":2,"SCM":2,"QAQC_MFG":1,"QAQC_SW":1},
    "상관":{"BRAND_PR":2,"PM_PRODUCT":2,"UX":1,"CONTENT":1},
    "정재":{"FIN_ACC":3,"OPS":2,"PUBLIC":1},
    "편재":{"SALES_B2B":3,"FIN_IB":2,"SCM":1,"BRAND_PR":1},
    "정관":{"PUBLIC":3,"STRATEGY":2,"QAQC_SW":1,"LAW":1},
    "편관":{"SECURITY":3,"LAW":2,"PUBLIC":1,"HSE":1},
    "정인":{"EDU":3,"DATA_SCI":1,"PUBLIC":1,"RND":1},
    "편인":{"DATA_SCI":3,"ML_ENG":2,"LAW":1,"RND":1},
}

# 4) 용희신 조합(네 judge_yongshin_simple의 '용희신' 값) → 가중치
MAP_BY_YONGSHIN_COMBO = {
    "인비":{"EDU":2,"COUNSEL":2,"DATA_SCI":1,"PM_PRODUCT":1},
    "재관":{"FIN_ACC":2,"FIN_IB":2,"PUBLIC":1,"SECURITY":1},
    "식재":{"BRAND_PR":2,"SALES_B2B":2,"PM_PRODUCT":1,"CONTENT":1},
    "식관":{"SALES_B2B":2,"SECURITY":2,"PUBLIC":1},
    "인식":{"EDU":2,"DATA_SCI":2,"UX":1,"RND":1},
    "재인":{"FIN_ACC":2,"DATA_SCI":1,"STRATEGY":1},
    "식재관":{"SALES_B2B":2,"BRAND_PR":1,"SECURITY":1,"PUBLIC":1},  # 신약 기구신 패널티 기준용
}

# === 본원/다수 오행(타고난 기질) 직군 보너스: CLUSTERS 키 기준 ===
NATIVE_ELEMENT_CAREERS = {
    "목": {
        "EDU": 1.2, "COUNSEL": 1.1, "UX": 0.8, "PM_PRODUCT": 0.4,
        "BRAND_PR": 0.5, "SALES_B2B": 0.5, "CSM": 0.5, "PUBLIC": 0.3
    },
    "화": {
        "BRAND_PR": 0.9, "SALES_B2B": 0.8, "CONTENT": 0.7,
        "PM_PRODUCT": 0.5, "GAME_DESIGN": 0.5, "EDU": 0.4
    },
    "토": {
        "OPS": 1.0, "SCM": 0.9, "QAQC_MFG": 0.7, "FIN_ACC": 0.6,
        "PUBLIC": 0.6, "PM_CONS": 0.6
    },
    "금": {
        "LAW": 1.0, "SECURITY": 0.9, "QAQC_SW": 0.7, "HSE": 0.6,
        "FIN_ACC": 0.6, "BACKEND": 0.4
    },
    "수": {
        "DATA_SCI": 0.9, "ML_ENG": 0.8, "DS_ENG": 0.7, "RND": 0.7,
        "BACKEND": 0.6, "COUNSEL": 0.6, "STRATEGY": 0.5
    }
}

# === CLUSTERS 태그 자동 보강: 오행/십성그룹 정규 태그 주입 ===
def _ensure_set_tags():
    for k,v in CLUSTERS.items():
        v.setdefault("tags", [])
        if not isinstance(v["tags"], set):
            v["tags"] = set(v["tags"])

def suggest_majors_for_job(job_key, elem_counter, judge_result, topn=6):
    """직업군에 기본 연결된 전공 + 오행 힌트(용신/다수오행)로 가볍게 재정렬"""
    base = list(JOB_TO_MAJORS.get(job_key, []))
    weights = {m: 1.0 for m in base}

    # 용신오행 우선 가점
    for e in (judge_result.get("용희신오행") or []):
        for m in MAP_ELEMENT_TO_MAJORS.get(e, []):
            weights[m] = weights.get(m, 0) + 0.6

    # 다수 오행(원국에서 개수 가장 큰 오행) 가점
    if elem_counter:
        dom_elem = max(elem_counter, key=elem_counter.get)
        for m in MAP_ELEMENT_TO_MAJORS.get(dom_elem, []):
            weights[m] = weights.get(m, 0) + 0.3

    ranked = sorted(weights.items(), key=lambda x: x[1], reverse=True)
    majors = [MAJOR_CATALOG.get(k, k) for k,_ in ranked[:topn]]
    return majors

def augment_clusters_with_canonical_tags():
    _ensure_set_tags()

    # 1) 오행 → 해당 직업들에 오행 태그 부여
    for elem, job_map in MAP_BY_ELEMENT.items():
        for job_key in job_map.keys():
            if job_key in CLUSTERS:
                CLUSTERS[job_key]["tags"].add(elem)  # '목','화','토','금','수'

    # 2) 십성(10개) → 5그룹(비/인/재/관/식)으로 변환 후 태그 부여
    sips_to_group = {
        "비견":"비","겁재":"비","식신":"식","상관":"식",
        "편재":"재","정재":"재","편관":"관","정관":"관",
        "편인":"인","정인":"인",
    }
    for sip, job_map in MAP_BY_TENGOD_10.items():
        g = sips_to_group.get(sip)
        if not g:
            continue
        for job_key in job_map.keys():
            if job_key in CLUSTERS:
                CLUSTERS[job_key]["tags"].add(g)     # '비','인','재','관','식'

    # 3) 용희신 조합 → 구성 태그('재','관','인','식','비')를 직업에 부여
    def _combo_to_tags(c):
        return set(list(c)) if c else set()

    for combo, job_map in MAP_BY_YONGSHIN_COMBO.items():
        for job_key in job_map.keys():
            if job_key in CLUSTERS:
                CLUSTERS[job_key]["tags"] |= _combo_to_tags(combo)

# 호출: 모듈 로드 시 1번만
augment_clusters_with_canonical_tags()


# 5) 스코어러
def recommend_jobs(myeongpan, elem_counter, sip_counter, judge_result, topk=10):
    """
    elem_counter: 오행 카운트
    sip_counter : 십성(10개) 카운트(일간 제외 로직 반영)
    judge_result: judge_yongshin_simple 결과(dict)
    """
    scores = {k:0.0 for k in CLUSTERS.keys()}

    # A) 오행 기여(정규화)
    total_e = sum(elem_counter.get(e,0) for e in ["목","화","토","금","수"]) or 1
    for e, cnt in elem_counter.items():
        w = cnt / total_e
        for c, add in MAP_BY_ELEMENT.get(e, {}).items():
            scores[c] += w * add

    # B) 십성(10개) 기여(정규화)
    ALL_SIPS_10 = ["비견","겁재","식신","상관","편재","정재","편관","정관","편인","정인"]
    total_s = sum(sip_counter.get(s,0) for s in ALL_SIPS_10) or 1
    for s in ALL_SIPS_10:
        cnt = sip_counter.get(s,0)
        if cnt <= 0:
            continue
        w = cnt / total_s
        for c, add in MAP_BY_TENGOD_10.get(s, {}).items():
            scores[c] += w * add

    # C) 용희신 조합 보정(+), 기구신 조합 페널티(-)
    yong_combo = judge_result.get("용희신")
    kigu_combo = judge_result.get("기구신")
    for c, add in MAP_BY_YONGSHIN_COMBO.get(yong_combo, {}).items():
        scores[c] += add * 1.0
    for c, sub in MAP_BY_YONGSHIN_COMBO.get(kigu_combo, {}).items():
        scores[c] -= sub * 0.7  # 페널티 계수

    # D) 용희신/기구신 오행 보정
    for e in (judge_result.get("용희신오행") or []):
        for c, add in MAP_BY_ELEMENT.get(e, {}).items():
            scores[c] += add * 0.5
    for e in (judge_result.get("기구신오행") or []):
        for c, sub in MAP_BY_ELEMENT.get(e, {}).items():
            scores[c] -= sub * 0.4

    # E) 월지 십성 보너스 (격국 반영)
    _, wolji = myeongpan["월주"]
    ilgan = myeongpan["일주"][0]
    wolji_sip = get_sipseong(ilgan, wolji, is_gan=False)
    for c, add in MAP_BY_TENGOD_10.get(wolji_sip, {}).items():
        scores[c] += add * 0.6

    # F) 본원/다수 오행(타고난 기질) 보너스
    day_elem = element_map[myeongpan["일주"][0]]
    dominant_elem = max(elem_counter, key=elem_counter.get)
    native_elems = {day_elem, dominant_elem}
    for elem in native_elems:
        qty = elem_counter.get(elem, 0)
        base = 0.25 if qty <= 2 else (0.4 if qty == 3 else 0.6)
        damp = 1.0
        if elem in (judge_result.get("기구신오행") or []):
            damp *= 0.6
            if qty >= 4:
                damp *= 0.8
        bonus = base * damp
        for job, w in NATIVE_ELEMENT_CAREERS.get(elem, {}).items():
            if job in scores:
                scores[job] += bonus * w

    # G) 용신 오행 부재 페널티
    for elem in (judge_result.get("용희신오행") or []):
        if elem_counter.get(elem, 0) == 0:
            for job, w in MAP_BY_ELEMENT.get(elem, {}).items():
                if job in scores:
                    scores[job] -= 0.32 * w

    # H) 타고난 기질 × 용신 프레임 시너지
    native_tags = set()
    if day_elem == "목" or dominant_elem == "목":
        native_tags.update({"관", "재"})
    if day_elem == "화" or dominant_elem == "화":
        native_tags.update({"재"})
    if day_elem == "수" or dominant_elem == "수":
        native_tags.update({"관", "인"})

    def _combo_to_tags(c):
        if not c: return set()
        return set(list(c))  # 예: '재관' -> {'재','관'}

    yong_tags = _combo_to_tags(judge_result.get("용희신"))

    for job, meta in CLUSTERS.items():
        tags = set(meta.get("tags", []))
        if native_tags and yong_tags and (tags & native_tags) and (tags & yong_tags):
            scores[job] += 0.25

    # I) 지지(동물) 기반 가중치: 일지(본인) 최우선, 나머지는 보조
    branches = {
        "년주": myeongpan["년주"][1],
        "월주": myeongpan["월주"][1],
        "일주": myeongpan["일주"][1],
        "시주": myeongpan["시주"][1],
    }
    for lbl, br in branches.items():
        w = DAY_BRANCH_WEIGHT if lbl == "일주" else OTHER_BRANCH_WEIGHT
        for job, add in BRANCH_JOB_HINTS.get(br, {}).items():
            if job in scores:
                scores[job] += w * add


    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:topk]

def explain_job_reason(job_key, judge_result, elem_counter, wolji_sip, day_elem=None, dominant_elem=None, day_branch=None):
    reasons = []
    yh = judge_result.get("용희신오행") or []
    kg = judge_result.get("기구신오행") or []
    if yh: reasons.append(f"용신오행 {','.join(yh)} 강점 반영")
    if kg: reasons.append(f"기신오행 {','.join(kg)} 조절")
    yc = judge_result.get("용희신")
    if yc: reasons.append(f"용신 조합 {yc}")
    if wolji_sip: reasons.append(f"월지 십성 {wolji_sip} 보너스")

    # 본원/다수 오행
    if day_elem:
        reasons.append(f"타고난 오행 {day_elem} 활용")
    if dominant_elem and dominant_elem != day_elem:
        reasons.append(f"다수 오행 {dominant_elem} 강점")

    # 일지(동물) 근거
    if day_branch and day_branch in BRANCH_ANIMAL_TRAITS:
        animal_name = BRANCH_ANIMAL_TRAITS[day_branch][0]
        reasons.append(f"일지 {animal_name} 성향 반영")

    # 용신 부재 알림(있을 때만)
    lack = [e for e,cnt in elem_counter.items() if cnt==0 and e in (yh or [])]
    if lack:
        reasons.append(f"용신오행 {','.join(lack)} 부재(보완 필요)")

    return " · ".join(reasons[:4])



# ======================
# 기질 요약 함수
# ======================
def classify_count(value):
    if value >= 3: return "과다"
    elif value >= 1: return "적당"
    else: return "부족"

def summarize_traits(myeongpan, elem_counter, elem_to_group, sip_counter, judge_result, shinsals, indiv_sips):
    summary = {}

    # 오행분석
    summary["오행분석"] = {}
    for elem in ["목","화","토","금","수"]:
        state = classify_count(elem_counter.get(elem,0))
        summary["오행분석"][f"{elem}({state})"] = OHAENG_TRAITS[elem][state]

    # 일지(본인) 동물 성향
    day_branch = myeongpan["일주"][1]
    animal, trait = BRANCH_ANIMAL_TRAITS.get(day_branch, ("-", ""))
    summary["일지(동물)성향(본인)"] = f"{animal}({day_branch}): {trait}"

    # 전체 지지 성향 (년/월/일/시)
    all_branch_traits = []
    for lbl in ["년주","월주","일주","시주"]:
        b = myeongpan[lbl][1]
        a, t = BRANCH_ANIMAL_TRAITS.get(b, ("-", ""))
        tag = "본인" if lbl=="일주" else lbl
        all_branch_traits.append(f"{tag}:{a}({b})")
    summary["지지(동물)성향(전체)"] = all_branch_traits

    # 십성분석 (5그룹)
    summary["십성분석"] = {}

    # 개별 십성 카운트를 5그룹으로 합산
    group_counts = {g: 0 for g in ["비겁", "인성", "재성", "관성", "식상"]}
    for sip, cnt in sip_counter.items():
        g = SIPS_TO_GROUP.get(sip)
        if g:
            group_counts[g] += cnt

    # 과다/적당/부족 판정
    for g in ["비겁", "인성", "재성", "관성", "식상"]:
        state = classify_count(group_counts[g])  # 3↑ 과다, 1~2 적당, 0 부족
        summary["십성분석"][f"{g}({state})"] = SIPSUNG_TRAITS[g][state]


    # 개별 십성 기질 (중복 제거)
    unique_sips = set(indiv_sips)   # 중복 제거
    summary["개별십성기질"] = []
    for s in unique_sips:
        if s in INDIVIDUAL_SIPSUNG_TRAITS:
            summary["개별십성기질"].append(f"{s}: {INDIVIDUAL_SIPSUNG_TRAITS[s]}")


    # 신강/신약
    summary["신강신약"] = SHINKANG_TRAITS[judge_result["판정"]]

    # 용희신/기구신 (오행)
    yh_elems = judge_result.get("용희신오행")
    gg_elems = judge_result.get("기구신오행")

    # 문자열이면 리스트로 변환
    if isinstance(yh_elems, str):
        yh_elems = [yh_elems]
    elif yh_elems is None:
        yh_elems = []

    if isinstance(gg_elems, str):
        gg_elems = [gg_elems]
    elif gg_elems is None:
        gg_elems = []

    summary["용희신/기구신(오행)"] = []
    for e in yh_elems:
        trait = YONGGI_TRAITS["용희신"].get(e, "")
        if trait:
            summary["용희신/기구신(오행)"].append(f"{e}용신: {trait}")

    for e in gg_elems:
        trait = YONGGI_TRAITS["기구신"].get(e, "")
        if trait:
            summary["용희신/기구신(오행)"].append(f"{e}기신: {trait}")


    # 십성 성격
    summary["용희신/기구신(십성)"] = []

    def add_traits(label, key, role):
        """label=용희신/기구신, key=재성·관성 등, role=용희신/기구신"""
        if key in YONGGI_SIPSUNG_TRAITS:
            trait = YONGGI_SIPSUNG_TRAITS[key].get(role)
            if trait:
                summary["용희신/기구신(십성)"].append(f"{key}-{label}: {trait}")

    # 용희신
    yh_group = judge_result.get("용희신")   # 예: "재관"
    if yh_group in COMBO_MAP:
        for g in COMBO_MAP[yh_group]:
            add_traits("용희신", g, "용희신")
    else:
        add_traits("용희신", yh_group, "용희신")

    # 기구신
    gg_group = judge_result.get("기구신")   # 예: "인비"
    if gg_group in COMBO_MAP:
        for g in COMBO_MAP[gg_group]:
            add_traits("기구신", g, "기구신")
    else:
        add_traits("기구신", gg_group, "기구신")

    # 십이신살
    summary["십이신살재능"] = []
    seen = set()
    for s in shinsals:
        if s in SHINSAL_TRAITS and s not in seen:
            summary["십이신살재능"].append(f"{s}: {SHINSAL_TRAITS[s]}")
            seen.add(s)

    return summary

# ======================
# 실행 예시
# ======================
if __name__ == "__main__":
    mp, df = build_myeongpan_df(1998, 3, 9, 13, 45, "여")
    print("🟦", df.attrs["제목"])
    print(df.to_string(index=False))

    # 🔹 여기서 먼저 분포와 판정 계산
    elem_counter, elem_to_group = summarize_element_distribution(mp)
    sip_counter = summarize_tengod_distribution(mp)
    judge_result = judge_yongshin_simple(mp)

    indiv_sips = [get_sipseong(mp["일주"][0], mp[label][0], True) for label in ["년주","월주","시주"]]
    indiv_sips += [get_sipseong(mp["일주"][0], mp[label][1], False) for label in ["년주","월주","시주"]]

    shinsals = []
    for label in ["년주","월주","일주","시주"]:
        shinsals.append(get_shinsal(mp["년주"][1], mp[label][1]))
        shinsals.append(get_shinsal(mp["일주"][1], mp[label][1]))

    # 🔹 이제 출력 가능
    print("\n🟦 오행분포")
    print("  ".join([
        f"{e}({elem_to_group[e]}):{elem_counter.get(e,0)}"
        for e in ["목","화","토","금","수"]
    ]))

    print("\n🟦 십성분포")
    ALL_SIPS = ["비견","겁재","식신","상관","편재","정재","편관","정관","편인","정인"]
    print("  ".join([f"{s}:{sip_counter.get(s,0)}" for s in ALL_SIPS]))

    print("\n🟦 신강/신약 및 용신 판별")
    print(f"점수: {judge_result['점수']} → {judge_result['판정']}")
    print(f"인성:{judge_result['인성점수']} 비겁:{judge_result['비겁점수']} 재성:{judge_result['재성점수']} 관성:{judge_result['관성점수']} 식상:{judge_result['식상점수']}")
    print(f"용희신 : {judge_result['용희신']}")
    print(f"기구신 : {judge_result['기구신']}")

    gegguk10 = judge_gegguk_10(mp)
    print("\n🟦 격국(월지 십성 기준)")
    print(f"월지 십성: {gegguk10['월지십성']}")
    print(f"격국: {gegguk10['격국']}")
    print(f"성격: {gegguk10['성격']}")

    traits_summary = summarize_traits(mp, elem_counter, elem_to_group, sip_counter, judge_result, shinsals, indiv_sips)

    print("\n🟦 기질 요약")
    for category, result in traits_summary.items():
        print(f"[ {category} ]")
        if isinstance(result, dict):
            for k,v in result.items():
                print(f"- {k}: {v}")
        elif isinstance(result, list):
            for r in result:
                print(f"- {r}")
        else:
            print(result)
        print()

       # 🔵 직업군 추천 (상위 10)
    print("\n🟦 직업 추천 (상위)")
    ranked = recommend_jobs(mp, elem_counter, sip_counter, judge_result, topk=10)

    # 설명용 컨텍스트(월지 십성, 본원오행, 다수오행)
    ilgan = mp["일주"][0]
    day_elem = element_map[ilgan]
    dominant_elem = max(elem_counter, key=elem_counter.get)
    _, wolji = mp["월주"]
    wolji_sip = get_sipseong(ilgan, wolji, is_gan=False)
    day_branch = mp["일주"][1]

    for key, sc in ranked:
        meta = CLUSTERS[key]
        tags = ",".join(sorted(set(meta.get("tags", []))))
        reason = explain_job_reason(
            key, judge_result, elem_counter, wolji_sip,
            day_elem=day_elem, dominant_elem=dominant_elem, day_branch=day_branch
        )
        print(f"- {meta['name']} | 점수:{sc:.2f} | 태그:{tags} | 근거: {reason}")

    majors = suggest_majors_for_job(key, elem_counter, judge_result, topn=6)
    print(f"  · 어울리는 전공: {', '.join(majors)}")



🟦 출생: 여 / 1998년 3월 9일 13시 45분
지점 천간(십성) 지지(십성) 십이운성 십이신살(년주기준) 십이신살(일주기준)
년주 무토(정재) 인목(겁재)   제왕         지살        망신살
월주 을목(비견) 묘목(비견)   건록         연살        장성살
일주 을목(본원) 묘목(비견)   건록         연살        장성살
시주 계수(편인) 미토(편재)    양        반안살        화개살

🟦 오행분포
목(비겁):5  화(식상):0  토(재성):2  금(관성):0  수(인성):1

🟦 십성분포
비견:3  겁재:1  식신:0  상관:0  편재:1  정재:1  편관:0  정관:0  편인:1  정인:0

🟦 신강/신약 및 용신 판별
점수: 70 → 신강
인성:5 비겁:80 재성:15 관성:0 식상:0
용희신 : 재관
기구신 : 인비

🟦 격국(월지 십성 기준)
월지 십성: 비견
격국: 건록격
성격: 자기주장 강하고 독립심·주도성 뚜렷. 동료와 협력도 가능하나 과하면 고집·충돌.

🟦 기질 요약
[ 오행분석 ]
- 목(과다): 고집이 강하고 자기 주장 강함, 성급하고 공격적이 될 수 있음. 계획은 크지만 실천이 부족할 수 있음.
- 화(부족): 활력이 부족하고 소극적이며 기회 포착이 늦음. 의욕과 열정이 잘 드러나지 않음.
- 토(적당): 성실하고 책임감 강함, 안정적이고 신뢰할 수 있는 성격. 중재와 조화 능력 뛰어남.
- 금(부족): 결단력 부족, 소심하고 우유부단. 자기 방어가 약해 타인에게 휘둘림.
- 수(적당): 지혜롭고 사려 깊음. 융통성이 있고 상황에 적응 잘함. 사고가 깊고 창의력 풍부.

[ 일지(동물)성향(본인) ]
토끼(묘): 온화·조화·미감. 관계 중재·교육·상담/디자인 강점. 과하면 우유부단.

[ 지지(동물)성향(전체) ]
- 년주:호랑이(인)
- 월주:토끼(묘)
- 본인:토끼(묘)
- 시주:양(미)

[ 십성분석 ]
- 비겁(과다): 자기주장이 강하고 독립심이 지나쳐 협력이 부족. 고집

In [ ]:
import json

def export_as_json(filename, myeongpan, elem_counter, sip_counter, judge_result,
                   traits_summary, ranked, majors):
    ilgan = myeongpan["일주"][0]
    day_elem = element_map[ilgan]
    dominant_elem = max(elem_counter, key=elem_counter.get)
    _, wolji = myeongpan["월주"]
    wolji_sip = get_sipseong(ilgan, wolji, is_gan=False)
    day_branch = myeongpan["일주"][1]

    data = {
        "출생정보": {
            "성별": myeongpan.get("성별"),
            "생년월일시": df.attrs.get("제목", "")
        },
        "명반": {
            "년주": myeongpan["년주"],
            "월주": myeongpan["월주"],
            "일주": myeongpan["일주"],
            "시주": myeongpan["시주"]
        },
        "오행분포": dict(elem_counter),
        "십성분포": dict(sip_counter),
        "신강신약": judge_result,
        "격국": judge_gegguk_10(myeongpan),
        "기질요약": traits_summary,
        "직업추천": []
    }

    # 직업 추천 + 근거
    for key, score in ranked:
        meta = CLUSTERS[key]
        tags = ",".join(sorted(set(meta.get("tags", []))))
        reason = explain_job_reason(
            key, judge_result, elem_counter, wolji_sip,
            day_elem=day_elem, dominant_elem=dominant_elem, day_branch=day_branch
        )
        data["직업추천"].append({
            "직업": meta["name"],
            "점수": round(score, 2),
            "태그": tags,
            "근거": reason
        })

    # 추천 전공
    data["추천전공"] = majors

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ JSON 저장 완료 → {filename}")


# 실행 예시
export_as_json(
    "result.json",
    mp, elem_counter, sip_counter, judge_result,
    traits_summary, ranked, majors
)

✅ JSON 저장 완료 → result.json
